In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from researchlib.single_import import *

In [3]:
model = builder([
    Reshape((-1, 784)),
    BinarizeLinear(784, 512),
    nn.BatchNorm1d(512),
    nn.Hardtanh(),
    BinarizeLinear(512, 256),
    nn.BatchNorm1d(256),
    nn.Hardtanh(),
    BinarizeLinear(256, 128),
    nn.BatchNorm1d(128),
    nn.Hardtanh(),
    nn.Linear(128, 10)  
])

In [4]:
train_loader = FromPublic('mnist', 'train', batch_size=128, num_workers=4)
test_loader = FromPublic('mnist', 'test', batch_size=128, num_workers=4)

In [5]:
runner = Runner(model, train_loader, test_loader, 'adam', 'focal')

In [6]:
for _ in range(3): runner.fit_onecycle(5e-2, mixup_alpha=0.4, callbacks=[Binarized()])

0.7402615964412689
0.9442

Test set: Average loss: 0.2429



0.7945199867884318
0.9524

Test set: Average loss: 0.2204



0.7947078875223795
0.9556

Test set: Average loss: 0.2077

